In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts, dim
from holoviews.operation import histogram
hv.extension('bokeh')
from bokeh.resources import INLINE

import importlib
import processing_parameters
import datetime

import functions_bondjango as bd
import functions_plotting as fp
import functions_loaders as fl
import pandas as pd
import numpy as np
import paths


In [ ]:
# set up the figure config
importlib.reload(fp)
importlib.reload(processing_parameters)
# define the target saving path
save_path = os.path.join(paths.figures_path, 'Vis_ethogram')

# define the printing mode
save_mode = True
# define the target document
target_document = 'paper'
# set up the figure theme
fp.set_theme()
# load the label dict
label_dict = processing_parameters.label_dictionary
variable_list = processing_parameters.variable_list

In [ ]:
importlib.reload(processing_parameters)
importlib.reload(fl)

search_string = processing_parameters.ethogram_string

queries = bd.query_database('analyzed_data', search_string)
path = [queries[0]['analysis_path']]
    
raw_data, _, _  = fl.load_preprocessing(path, queries)

raw_data = raw_data[0]
print(raw_data.shape)
print(raw_data.columns)

# Ethogram plot 

In [ ]:
# plot an ethogram from the selected experiment

importlib.reload(fp)
# define the target range
target_range = np.arange(200)
data = raw_data.iloc[target_range, :]
# separate cells and other parameters
cell_columns = [el for el in data.columns if 'cell' in el]
not_cell_columns = [el for el in data.columns if 'cell' not in el]
calcium_data = data.loc[:, cell_columns]
behavior_data = data.loc[:, not_cell_columns]

# get the data
current_calcium = calcium_data
current_behavior = behavior_data.loc[:, variable_list]
time = data.loc[:, 'time_vector']

# generate the plots
# calcium_plot = hv.Image((x, y, current_calcium.to_numpy().T), 
#                         kdims=['Time', 'Cells'], vdims=['Activity'])
calcium_plot = hv.Raster(current_calcium.to_numpy().T, kdims=['Time', 'Cells'], vdims=['Activity'])
calcium_plot.opts(frame_width=800, frame_height=300, xaxis=None, colorbar=True, border=0, ylabel='Cells', yticks=np.arange(50, 200, 50))
calcium_plot.opts(cmap='Reds')
# calcium_plot.opts(width=800, height=200, color='k')

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'Ethogram_calcium')) + '.png')
# # save the figure
calcium_plot = fp.save_figure(calcium_plot, save_name, fig_width=5.4, dpi=1200, fontsize='small', target='save')
ethogram = []
ethogram.append(calcium_plot)
# for idx, el in enumerate(variable_list):
#     beh_plot = hv.Curve((time, current_behavior.loc[:, el]), kdims=['Time'], vdims=el)
#     beh_plot.opts(width=800, height=50, xaxis=None, yaxis=None, show_frame=False)
#     if idx == len(variable_list) - 1:
#         beh_plot.opts(show_frame=False)

yticks = [(idx+0.5, label_dict[el]) for idx, el in enumerate(variable_list)]
xticks = [(int(el), f'{time[el]:0.0f}') for el in np.arange(50, 200, 50)]

#     ethogram.append(beh_plot)
behavior_matrix = current_behavior.to_numpy().T
# behavior_matrix /= behavior_matrix.max(axis=0)
behavior_matrix = (behavior_matrix - behavior_matrix.min(axis=0))/(behavior_matrix.max(axis=0) - behavior_matrix.min(axis=0))
behavior_plot = hv.Raster(behavior_matrix, kdims=['Time (s)', 'Feature'], vdims=['Value'])
behavior_plot.opts(frame_width=800, frame_height=400, colorbar=True, border=0, cmap='PuBu', ylabel='', yticks=yticks, tools=['hover'], xticks=xticks)

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'Ethogram_behavior')) + '.png')
# # save the figure
fig = fp.save_figure(behavior_plot, save_name, fig_width=5.4, dpi=1200, fontsize='small', target='save')

ethogram.append(behavior_plot)
# behavior_plot.opts(width=800, height=200, color='k')
ethogram = hv.Layout(ethogram).cols(1)
ethogram


# final_plot = (calcium_plot+behavior_plot).cols(1)
# final_plot

In [ ]:
# Plot the target cell

# define the target cell
target_cell = 42
# define the target behavior
target_behavior = ['cricket_0_mouse_distance', 'cricket_0_delta_heading']
# format the behavior labels
formatted_labels = []
for label in target_behavior:
    new_label = label.replace('cricket_0', 'cricket_-')
    new_label = new_label.split('_')
    new_label = ' '.join([el.capitalize() for el in new_label])
    formatted_labels.append(new_label)

# separate cells and other parameters
cell_columns = [el for el in data.columns if 'cell' in el]
not_cell_columns = [el for el in data.columns if 'cell' not in el]
calcium_data = data.loc[:, cell_columns]
behavior_data = data.loc[:, not_cell_columns]

# get the data
current_calcium = calcium_data.iloc[:, target_cell]
current_behavior = behavior_data.loc[:, target_behavior]
time = data.loc[:, 'time_vector']

# generate the plots
calcium_plot = hv.Curve((time, current_calcium), kdims=['Time'], vdims=['Activity'])
calcium_plot.opts(width=800, height=200, color='k')

plot_list = []
plot_list.append(calcium_plot)
for idx, label in enumerate(target_behavior):
    behavior_plot = hv.Curve((time, current_behavior.loc[:, target_behavior[idx]]), kdims=['Time'], vdims=formatted_labels[idx])
    behavior_plot.opts(width=800, height=200, color='k')
    plot_list.append(behavior_plot)


plot_parts = hv.Layout(plot_list).cols(1).opts(width=800)

save_name = os.path.join(save_path, '_'.join((target_document, 'Diagram')) + '.svg')
hv.save(plot_parts, save_name, backend='matplotlib')

plot_parts
# (calcium_plot+behavior_plot).cols(1)



In [ ]:
hv.help(hv.save)

In [ ]:
# Plot a full ethogram

# define the target behavior
target_behavior = ['cricket_0_mouse_distance', 'cricket_0_delta_heading', 
                   'mouse_speed', 'cricket_0_speed']
# # format the behavior labels
# formatted_labels = []
# for label in target_behavior:
#     new_label = label.replace('cricket_0', 'cricket_-')
#     new_label = new_label.split('_')
#     new_label = ' '.join([el.capitalize() for el in new_label])
#     formatted_labels.append(new_label)

# separate cells and other parameters
cell_columns = [el for el in data.columns if 'cell' in el]
not_cell_columns = [el for el in data.columns if 'cell' not in el]
calcium_data = data.loc[:, cell_columns]
behavior_data = data.loc[:, not_cell_columns]

# get the data
current_calcium = calcium_data
current_behavior = behavior_data.loc[:, target_behavior]
time = data.loc[:, 'time_vector']

x = time
y = np.array(np.arange(current_calcium.shape[1]))
# generate the plots
calcium_plot = hv.Image((x, y, current_calcium.to_numpy().T), 
                        kdims=['Time', 'Cells'], vdims=['Activity'])
calcium_plot = fp.format_figure(calcium_plot, width=800)
calcium_plot.opts(cmap='Purples')
# calcium_plot.opts(width=800, height=200, color='k')
behavior_plot = []
for idx, el in enumerate(target_behavior):
    beh_plot = hv.Curve((time, current_behavior.loc[:, el]), kdims=['Time'], vdims=el)
    beh_plot = fp.format_figure(beh_plot, width=800, height=100)
    if idx == len(target_behavior) - 1:
        beh_plot.opts(show_frame=False)
    
    behavior_plot.append(beh_plot)
# behavior_plot.opts(width=800, height=200, color='k')
behavior_plot = hv.Layout(behavior_plot).cols(1)
final_plot = (calcium_plot+behavior_plot).cols(1)
final_plot
# assemble the file name
# save_name = os.path.join(save_path, 'ethogram.png')
# save the figure
# hv.save(final_plot, save_name, dpi=600)